In [ ]:
# app.py
import pandas as pd, numpy as np, plotly.graph_objects as go
import networkx as nx

# st.set_page_config(page_title="Warehouse Twin", layout="wide")


In [ ]:
import pandas as pd, numpy as np, plotly.graph_objects as go
import networkx as nx
import math
from scipy.spatial import cKDTree

In [3]:
st_loc = pd.read_csv("Storage_Location.csv")
st_loc

,originalLocation,position,x,y,z
0,A-14-11,"368, 0, 1",368,0,1
1,A-14-12,"352, 0, 1",352,0,1
2,A-14-13,"336, 0, 1",336,0,1
3,A-14-21,"368, 0, 2",368,0,2
4,A-14-22,"352, 0, 2",352,0,2
...,...,...,...,...,...
2287,P-26-40,"403, 1350, 4",403,1350,4
2288,Q-25-30,"403, 1408, 3",403,1408,3
2289,Q-25-40,"403, 1408, 4",403,1408,4
2290,Q-26-30,"403, 1440, 3",403,1440,3


For Now we only work with z = 1 (first floor)

In [4]:
st_loc_1 = st_loc.loc[st_loc['z'] == 1, :].copy()
st_loc_1

,originalLocation,position,x,y,z
0,A-14-11,"368, 0, 1",368,0,1
1,A-14-12,"352, 0, 1",352,0,1
2,A-14-13,"336, 0, 1",336,0,1
8,A-16-11,"318, 0, 1",318,0,1
9,A-16-12,"302, 0, 1",302,0,1
...,...,...,...,...,...
2249,R-21-12,"152, 1440, 1",152,1440,1
2250,R-21-13,"136, 1440, 1",136,1440,1
2256,R-23-11,"118, 1440, 1",118,1440,1
2257,R-23-12,"102, 1440, 1",102,1440,1


In [5]:
# small minor changes on the "I" and "H" blocks so it is closer to the cross Aisle path (just for this prototype)
i_block_list_down = []
for i in range(13, 24, 2):
    i_block_list_down += [f"I-{i}-{j}" for j in [11,12,13]]

i_block_list_up = []
for i in range(14, 25, 2):
    i_block_list_up += [f"I-{i}-{j}" for j in [11,12,13]]

h_block_list = []
for i in range(2,11,2):
    if i < 9: h_block_list += [f"H-0{i}-{j}" for j in [11,12,13]]
    else: h_block_list += [f"H-{i}-{j}" for j in [11,12,13]]

st_loc_1.loc[st_loc_1["originalLocation"].isin(i_block_list_down), "y"] = st_loc_1.loc[st_loc_1["originalLocation"].isin(i_block_list_down), "y"] + 30
st_loc_1.loc[st_loc_1["originalLocation"].isin(i_block_list_up), "y"] = st_loc_1.loc[st_loc_1["originalLocation"].isin(i_block_list_up), "y"] - 30
st_loc_1.loc[st_loc_1["originalLocation"].isin(h_block_list), "y"] = st_loc_1.loc[st_loc_1["originalLocation"].isin(h_block_list), "y"] - 25

In [6]:
st_loc_1

,originalLocation,position,x,y,z
0,A-14-11,"368, 0, 1",368,0,1
1,A-14-12,"352, 0, 1",352,0,1
2,A-14-13,"336, 0, 1",336,0,1
8,A-16-11,"318, 0, 1",318,0,1
9,A-16-12,"302, 0, 1",302,0,1
...,...,...,...,...,...
2249,R-21-12,"152, 1440, 1",152,1440,1
2250,R-21-13,"136, 1440, 1",136,1440,1
2256,R-23-11,"118, 1440, 1",118,1440,1
2257,R-23-12,"102, 1440, 1",102,1440,1


In [7]:
nav = pd.read_csv("Support_Points_Navigation.csv", sep=";")
nav

,points_specified,labels
0,"(66.0, -29.0, 1.0)",LC-01
1,"(66.0, 61.0, 1.0)",LC-02
2,"(66.0, 151.0, 1.0)",LC-03
3,"(66.0, 241.0, 1.0)",LC-04
4,"(66.0, 331.0, 1.0)",LC-05
5,"(66.0, 421.0, 1.0)",LC-06
6,"(66.0, 511.0, 1.0)",LC-07
7,"(66.0, 631.0, 1.0)",LC-08
8,"(66.0, 751.0, 1.0)",LC-09
9,"(66.0, 841.0, 1.0)",LC-10


First we split the x y z from points column

In [8]:
for i, row in nav.iterrows():
    x_y_z = row['points_specified']
    x_y_z_list = x_y_z.strip("(").strip(")").split(",")
    nav.loc[i,"x"] = float(x_y_z_list[0].strip())
    nav.loc[i,"y"] = float(x_y_z_list[1].strip())
    nav.loc[i,"z"] = float(x_y_z_list[2].strip())

nav
    

,points_specified,labels,x,y,z
0,"(66.0, -29.0, 1.0)",LC-01,66.0,-29.0,1.0
1,"(66.0, 61.0, 1.0)",LC-02,66.0,61.0,1.0
2,"(66.0, 151.0, 1.0)",LC-03,66.0,151.0,1.0
3,"(66.0, 241.0, 1.0)",LC-04,66.0,241.0,1.0
4,"(66.0, 331.0, 1.0)",LC-05,66.0,331.0,1.0
5,"(66.0, 421.0, 1.0)",LC-06,66.0,421.0,1.0
6,"(66.0, 511.0, 1.0)",LC-07,66.0,511.0,1.0
7,"(66.0, 631.0, 1.0)",LC-08,66.0,631.0,1.0
8,"(66.0, 751.0, 1.0)",LC-09,66.0,751.0,1.0
9,"(66.0, 841.0, 1.0)",LC-10,66.0,841.0,1.0


I need to add another center aisle because the center aisle has a far distance between the left and right block. which cause problem for creating edges in the next step.<Br>
I name them CCL and CCR. Same "y" but different "x"

In [9]:
cc_left = nav.loc[nav["labels"].str.startswith("CC"), :].copy()
cc_right = nav.loc[nav["labels"].str.startswith("CC"), :].copy()

# pushing right cc 12 meters right
cc_right["labels"]=cc_right["labels"].str.split("-").str[0] + "R-" + cc_right["labels"].str.split("-").str[1]
cc_right["x"] = cc_right["x"] + 12

# pushing left cc 14 meters left
cc_left["labels"]=cc_left["labels"].str.split("-", expand=True)[0] + "L-" + cc_left["labels"].str.split("-", expand=True)[1]
cc_left["x"] = cc_left["x"] - 14

# remove the CC to add CCR and CCL
nav = nav.loc[~nav["labels"].str.startswith("CC"), :]

# adding all together
nav = pd.concat([nav, cc_left, cc_right], axis=0)

I want to mirror the Shelves of the Warehouse to the closest Aisle So that i can connect the Shelves to the Aisle with an edge. 

In [10]:
def find_closest(numbers, target) -> float: 
    """
    Finds the number in a list closest to a target value.

    Args:
        numbers: A list of numbers.
        target: The target number.

    Returns:
        The number in 'numbers' closest to 'target'.
    """
    return min(numbers, key=lambda x: abs(x - target))

# list of the y coordinate of the Aisle and cross Aisle of the warehouse
nav_y_list = list(nav["y"])

# add new columns for new values

for i, row in st_loc_1.iterrows():
    # we name the columns
    st_loc_1.loc[i,"location_on_aisle"] = row["originalLocation"] + "-T"

    # we will find the closes "y" to the Shelf
    st_loc_1.loc[i, "y_on_aisle"] = find_closest(nav_y_list, row["y"])
    
st_loc_1

,originalLocation,position,x,y,z,location_on_aisle,y_on_aisle
0,A-14-11,"368, 0, 1",368,0,1,A-14-11-T,-29.0
1,A-14-12,"352, 0, 1",352,0,1,A-14-12-T,-29.0
2,A-14-13,"336, 0, 1",336,0,1,A-14-13-T,-29.0
8,A-16-11,"318, 0, 1",318,0,1,A-16-11-T,-29.0
9,A-16-12,"302, 0, 1",302,0,1,A-16-12-T,-29.0
...,...,...,...,...,...,...,...
2249,R-21-12,"152, 1440, 1",152,1440,1,R-21-12-T,1471.0
2250,R-21-13,"136, 1440, 1",136,1440,1,R-21-13-T,1471.0
2256,R-23-11,"118, 1440, 1",118,1440,1,R-23-11-T,1471.0
2257,R-23-12,"102, 1440, 1",102,1440,1,R-23-12-T,1471.0


In [11]:
# remove the duplicates in "location_on_aisle" because each shelf has two same twin on cross aisle which has the same "x" and "y_on_aisle"
location_on_aisle_unique = st_loc_1.groupby(["x", "y_on_aisle"]).min()["location_on_aisle"].to_list()

# put duplicates as None in the main dataframe
st_loc_1.loc[~st_loc_1["location_on_aisle"].isin(location_on_aisle_unique), "location_on_aisle"] = None

st_loc_1

,originalLocation,position,x,y,z,location_on_aisle,y_on_aisle
0,A-14-11,"368, 0, 1",368,0,1,A-14-11-T,-29.0
1,A-14-12,"352, 0, 1",352,0,1,A-14-12-T,-29.0
2,A-14-13,"336, 0, 1",336,0,1,A-14-13-T,-29.0
8,A-16-11,"318, 0, 1",318,0,1,A-16-11-T,-29.0
9,A-16-12,"302, 0, 1",302,0,1,A-16-12-T,-29.0
...,...,...,...,...,...,...,...
2249,R-21-12,"152, 1440, 1",152,1440,1,R-21-12-T,1471.0
2250,R-21-13,"136, 1440, 1",136,1440,1,R-21-13-T,1471.0
2256,R-23-11,"118, 1440, 1",118,1440,1,R-23-11-T,1471.0
2257,R-23-12,"102, 1440, 1",102,1440,1,R-23-12-T,1471.0


Let's build our network graph

In [12]:
nodes_dict = {}

# first we build our nodes
for node in st_loc_1["originalLocation"]:
    nodes_dict[node] = {"x": float(st_loc_1.loc[st_loc_1["originalLocation"]==node, "x"].iloc[0]),
                        "y": float(st_loc_1.loc[st_loc_1["originalLocation"]==node, "y"].iloc[0])}

for node in st_loc_1["location_on_aisle"]:
    if node: # consider only available ones - we removed duplicates
        nodes_dict[node] = {"x": float(st_loc_1.loc[st_loc_1["location_on_aisle"]==node, "x"].iloc[0]),
                            "y": float(st_loc_1.loc[st_loc_1["location_on_aisle"]==node, "y_on_aisle"].iloc[0])}

for node in nav["labels"]:
    nodes_dict[node] = {"x": float(nav.loc[nav["labels"]==node, "x"].iloc[0]),
                        "y": float(nav.loc[nav["labels"]==node, "y"].iloc[0])}

nodes_dict

{'A-14-11': {'x': 368.0, 'y': 0.0},
 'A-14-12': {'x': 352.0, 'y': 0.0},
 'A-14-13': {'x': 336.0, 'y': 0.0},
 'A-16-11': {'x': 318.0, 'y': 0.0},
 'A-16-12': {'x': 302.0, 'y': 0.0},
 'A-16-13': {'x': 286.0, 'y': 0.0},
 'A-18-11': {'x': 268.0, 'y': 0.0},
 'A-18-12': {'x': 252.0, 'y': 0.0},
 'A-18-13': {'x': 236.0, 'y': 0.0},
 'A-20-11': {'x': 218.0, 'y': 0.0},
 'A-20-12': {'x': 202.0, 'y': 0.0},
 'A-20-13': {'x': 186.0, 'y': 0.0},
 'A-22-11': {'x': 168.0, 'y': 0.0},
 'A-22-12': {'x': 152.0, 'y': 0.0},
 'A-22-13': {'x': 136.0, 'y': 0.0},
 'A-24-11': {'x': 118.0, 'y': 0.0},
 'A-24-12': {'x': 102.0, 'y': 0.0},
 'A-24-13': {'x': 86.0, 'y': 0.0},
 'B-13-11': {'x': 368.0, 'y': 32.0},
 'B-13-12': {'x': 352.0, 'y': 32.0},
 'B-13-13': {'x': 336.0, 'y': 32.0},
 'B-14-11': {'x': 368.0, 'y': 90.0},
 'B-14-12': {'x': 352.0, 'y': 90.0},
 'B-14-13': {'x': 336.0, 'y': 90.0},
 'B-15-11': {'x': 318.0, 'y': 32.0},
 'B-15-12': {'x': 302.0, 'y': 32.0},
 'B-15-13': {'x': 286.0, 'y': 32.0},
 'B-16-11': {'x': 31

In [13]:
G = nx.Graph()


# let's have some helper functions:
def is_support(name: str) -> bool:
    return name.startswith(("LC", "CC", "RC"))

def is_twin(name: str) -> bool:
    return name.endswith("-T")

def dist(a, b, G):
    ax, ay = G.nodes[a]["x"], G.nodes[a]["y"]
    bx, by = G.nodes[b]["x"], G.nodes[b]["y"]
    return math.hypot(ax - bx, ay - by)

# ---1) first adding the nodes
for key, coordinate in nodes_dict.items():
    # defind the kind of node
    if is_twin(key): kind = "twin"
    elif is_support(key): kind = "support"
    else: kind = "shelf"
    # add the node
    G.add_node(key, x=coordinate["x"], y=coordinate["y"], kind=kind)


# ---- 2) connect "shelves" to the closest node which is their twin nodes on aisle

# explanation below on the next section
shelf_twin_nodes = [n for n in G.nodes]
shelf_twin_xy = [(G.nodes[n]["x"], G.nodes[n]["y"]) for n in shelf_twin_nodes]
tree = cKDTree(shelf_twin_xy)

# Loop over all shelf nodes, with both their index (i) and name (u)
for i, u in enumerate(shelf_twin_nodes):
    if u.endswith("-T"):
        # Find the 5 nearest neighbors of point i (including itself as the closest)
        dists, idxs = tree.query(shelf_twin_xy[i], k=5)
        # Loop over each neighbor index
        for j in idxs:
            if j == i:
                continue  # skip itself (we don’t want u–u edges)

            # Get the neighbor's name using the same index j
            v = shelf_twin_nodes[j]

            if not v.endswith("-T"):
                # Add the edge between u and v with distance as weight
                G.add_edge(u, v, weight=dist(u, v, G), kind="twin_link")
        

# ---3) Connect "support" and "twin" nodes to their two nearest neighbors, no duplicates

# Create a list of node names that are "support or twin nodes"
support_twin_nodes = [n for n in G.nodes if is_support(n) or is_twin(n)]
# Example result: ["LC-11", "LC-12", "CC-11", "A-24-13-T", B-22-12-T, ...]

# Create a list of coordinates for those nodes, in the *same order* as support_twin_nodes
support_twin_xy = [(G.nodes[n]["x"], G.nodes[n]["y"]) for n in support_twin_nodes]
# Example result: [(0,0), (0,10), (5,0), (5,10), ...]

# Build a KDTree for fast nearest-neighbor search on the coordinates
tree = cKDTree(support_twin_xy)
# Now we can ask "give me the closest 3 points to (x,y)" very quickly. -> we want 3 because it also consider the self point

# Keep a set of already-added edges, stored as sorted tuples, so we don't add duplicates
added = set()
# Example: if we add ("LC-11","CC-11"), we'll store ("CC-11","LC-11") in the set

# Loop over all support nodes, with both their index (i) and name (u)
for i, u in enumerate(support_twin_nodes):
    # Find the 3 nearest neighbors of point i (including itself as the closest)
    dists, idxs = tree.query(support_twin_xy[i], k=3)
    # Example: idxs might be [0, 1, 2] meaning "self + neighbor1 + neighbor2"
    # dists are the corresponding distances

    # Loop over each neighbor index
    for j in idxs:
        if j == i:
            continue  # skip itself (we don’t want u–u edges)

        # Get the neighbor's name using the same index j
        v = support_twin_nodes[j]
        # This works because support_nodes and support_xy have the same order!
        # i = index in support_nodes → support_xy[i] = its coordinates

        # Create a unique key for the edge, order-independent
        pair = tuple(sorted((u, v)))
        if pair in added:
            continue  # edge already exists, skip it

        # Optional: apply constraints (e.g. only same aisle group, or max distance)
        # if G.nodes[u]['group'] != G.nodes[v]['group']: continue
        if not v.startswith(("LC", "CCL", "CCR" "RC")):
            # Add the edge between u and v with distance as weight
            G.add_edge(u, v, weight=dist(u, v, G), kind="main_link")

        # Mark this edge as added, so we don't add it again later
        added.add(pair)


# We have to also connect the support nodes vertically
support_nodes = [n for n in G.nodes if is_support(n)]

for k in ["LC", "CCL", "CCR", "RC"]:
    for i in range(1, 17):
        if f"{k}-0{i}" in support_nodes or f"{k}-{i}" in support_nodes:
            if i < 9: G.add_edge(f"{k}-0{i}", f"{k}-0{i+1}", weight=dist(f"{k}-0{i}", f"{k}-0{i+1}", G), kind="support")
            elif i == 9: G.add_edge(f"{k}-0{i}", f"{k}-{i+1}", weight=dist(f"{k}-0{i}", f"{k}-{i+1}", G), kind="support")
            else: G.add_edge(f"{k}-{i}", f"{k}-{i+1}", weight=dist(f"{k}-{i}", f"{k}-{i+1}", G), kind="support")

# adding one last node and edge for dropping off the orders
G.add_node("Start", x=500, y=511, kind="support")
G.add_edge("CCR-07", "Start", weight=dist("CCR-07", "Start", G), kind="support")

Store the "G" our Networkx to import in our app.

In [14]:
import pickle

# Save
with open("warehouse_graph.pkl", "wb") as f:
    pickle.dump(G, f)

In [196]:
import plotly.graph_objects as go
    
def plot_graph(G, pos=None):
    if pos is None:
        pos = nx.spring_layout(G)
    # edges
    edge_x, edge_y = [], []
    for u, v in G.edges():
        x0, y0 = pos[u]; x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]
    edge_trace = go.Scatter(x=edge_x, y=edge_y, mode='lines',
                            line=dict(width=1), hoverinfo='none')

    # nodes (color by kind)
    kind_color = {'shelf':'#4e79a7','support':'#f28e2b','twin':'#f28e2b'}
    node_x, node_y, node_text, node_color = [], [], [], []
    for n, d in G.nodes(data=True):
        x, y = pos[n]
        node_x.append(x); node_y.append(y)
        node_text.append(f"{n}<br>({x:.2f},{y:.2f})<br>kind={d.get('kind','?')}")
        node_color.append(kind_color.get(d.get('kind','shelf'), '#9c9c9c'))

    node_trace = go.Scatter(x=node_x, y=node_y, mode='markers+text',
                            text=[str(n) for n in G.nodes()],
                            textposition='top center',
                            marker=dict(size=14, color=node_color),
                            hovertext=node_text, hoverinfo='text')

    fig = go.Figure([edge_trace, node_trace])
    # fig.update_yaxes(scaleanchor="x", scaleratio=1)   # keep real scale
    fig.update_layout(
        title="Warehouse Graph",                                # chart title
        xaxis=dict(scaleanchor="y", showgrid=False),# lock aspect ratio; hide grid
        yaxis=dict(showgrid=False),                 # hide grid
        showlegend=False,                            # hide legend for a cleaner look
        width=800,
        height=1200
    )
    fig.show()
    return fig

# Example
# G = nx.Graph()
# G.add_edge(1, 2)
pos = {n: (float(d["x"]), float(d["y"])) for n, d in G.nodes(data=True)}
fig = plot_graph(G, pos)


Our Nodes and Edges are now done.

In [188]:
pw = pd.read_csv("Picking_Wave.csv", sep=";")
# lets filter the picking wave data only on the warehouse node that we have.
nodes=[node for node in G.nodes]

pw_filtered = pw.loc[pw['locations'].str.strip().isin(nodes),:]

pw_filtered

,waveNumber,reference,Size (US),quantityToPick (units),locations,operator
0,43175,8N10W9,9.0,1,H-06-13,Operator_1
1,43175,8N10W9,9.0,1,H-06-13,Operator_1
2,43175,8N10W9,9.0,1,H-06-13,Operator_1
3,43175,8N10W9,9.0,1,H-06-13,Operator_1
4,43175,8N10W9,9.0,1,H-06-13,Operator_1
...,...,...,...,...,...,...
215177,33168,UHYJ3A,8.5,1,O-13-11,Operator_2
215179,33168,8E9F1P,10.0,1,M-13-11,Operator_2
215180,33168,8N10W9,15.0,1,M-04-13,Operator_2
215187,33168,RAUDQ6,11.0,1,K-16-12,Operator_2


In [189]:
# save pw_filtered
pw_filtered.to_csv("Picking_Wave_filtered.csv", index=False)

In [17]:
def nearest_neighbor_order(G, stops, start):
    """Greedy order: from current, go to the unvisited stop with smallest shortest-path distance."""
    unvisited = set(stops) - {start}
    order = [start]
    current = start
    while unvisited:
        # pick the next stop with minimal shortest-path length (by weight)
        nxt = min(unvisited, key=lambda v: nx.shortest_path_length(G, current, v, weight="weight"))
        order.append(nxt)
        unvisited.remove(nxt)
        current = nxt
    return order

def build_node_route(G, stop_nodes:list):
    """
    Return a flat list of node IDs that forms the complete walk visiting every stop in order.
    Example: stops = [DEPOT, S1, S2, DEPOT] -> route includes shortest path DEPOT->S1, then S1->S2, then S2->DEPOT.
    """
    route = []  # will accumulate node ids in walking order
    for a, b in zip(stop_nodes[:-1], stop_nodes[1:]):  # iterate consecutive pairs: (stop[i], stop[i+1])
        hop = nx.shortest_path(G, a, b, weight="weight")  
        if route:                     # if we already have nodes in the route...
            route.extend(hop[1:])     # append hop without its first node to avoid duplicating the join
        else:
            route.extend(hop)         # first hop: take all nodes
    return route                      


# stops = {"Start", S1, S2, S3, S4}
# visit_order = nearest_neighbor_order(G, stops, start="Start")
# route_nodes = build_node_route(G, visit_order + ["Start"])  # return to depot if you need

In [204]:
# connecting Picking_Wave data to customer orders data
co = pd.read_csv("Customer_Order.csv", sep=";")
pw = pd.read_csv("Picking_Wave_filtered.csv")
co.rename(columns={'Reference':'reference'}, inplace=True)
order_selected = 123619
co_order_filtered = co.loc[co["orderNumber"]== order_selected, ["orderNumber", "orderToCollect", "waveNumber", "operator", "reference", "quantity (units)"]]
co_pw_merged = pd.merge(left=pw[["waveNumber","locations", 'reference']], right=co_order_filtered, on=['waveNumber', 'reference'], how="right")
# co_order_filtered
co_pw_merged = co_pw_merged.groupby(["waveNumber", "locations", "reference", "orderNumber", "operator", "orderToCollect"])['quantity (units)'].max().reset_index()

paths = {}
for op in co_pw_merged['operator'].unique():
    co_pw_merged_filtered_by_op = co_pw_merged.loc[co_pw_merged['operator']==op, :]
    path = co_pw_merged_filtered_by_op.sort_values('orderToCollect')['locations']
    paths[op] = [p.strip() for p in path.values]

paths

{'Operator_6': ['I-05-13',
  'I-07-13',
  'I-20-11',
  'K-23-11',
  'K-23-11',
  'K-22-12',
  'K-22-12',
  'M-05-11',
  'M-05-11',
  'L-14-13',
  'L-14-13',
  'L-16-12',
  'M-16-12',
  'I-14-13',
  'I-14-12',
  'I-14-12',
  'I-14-13',
  'I-18-12',
  'H-06-11',
  'H-06-11',
  'H-06-11',
  'H-06-11',
  'H-10-11',
  'H-10-11'],
 'Operator_5': ['P-13-11',
  'M-03-13',
  'L-16-11',
  'L-16-11',
  'L-16-11',
  'M-06-11',
  'I-21-12',
  'I-23-12',
  'I-21-12',
  'I-23-12',
  'I-21-12',
  'I-23-12',
  'I-21-12',
  'I-23-12',
  'Q-09-12',
  'N-01-12',
  'N-01-12',
  'Q-09-12',
  'K-18-12',
  'K-16-11',
  'K-18-12',
  'K-16-11',
  'K-18-12',
  'K-16-11',
  'K-18-12',
  'K-16-11',
  'K-18-12',
  'K-16-11',
  'K-18-12',
  'K-16-11',
  'M-01-12',
  'M-01-12',
  'M-01-12']}

In [203]:
for op, path in paths.items():
    path.insert(0, 'Start')
    route_nodes = build_node_route(G, path + ["Start"])
    
route_nodes

['Start',
 'CCR-07',
 'CCL-07',
 'CCL-08',
 'CCL-09',
 'CCL-10',
 'CCL-11',
 'L-13-11-T',
 'L-13-12-T',
 'L-13-13-T',
 'L-15-11-T',
 'L-15-12-T',
 'L-15-13-T',
 'L-17-11-T',
 'L-17-12-T',
 'L-17-13-T',
 'L-18-13',
 'L-17-13-T',
 'L-17-12-T',
 'L-17-11-T',
 'L-15-13-T',
 'L-15-12-T',
 'L-15-11-T',
 'L-13-13-T',
 'L-13-12-T',
 'L-13-11-T',
 'CCL-11',
 'CCL-10',
 'K-13-11-T',
 'K-14-11',
 'K-13-11-T',
 'CCL-10',
 'CCR-10',
 'CCR-09',
 'CCR-08',
 'CCR-07',
 'Start']

In [ ]:
visit_order = nearest_neighbor_order(G, ["H-06-13", "L-24-12", "I-18-13"], start="Start")  

route_nodes = build_node_route(G, visit_order + ["Start"])  # return to depot if you need

route_nodes

Calculate total path for an order (picking wave)

In [199]:
def path_length(route_nodes, G):
    total = 0.0
    for i in range(len(route_nodes) - 1):
        total += dist(route_nodes[i], route_nodes[i+1], G)
    return total

length = path_length(route_nodes, G)
print("Total path length for this order picking is: ",length, "meter")

Total path length for this order picking is:  1650.0 meter


In [200]:
def visited_path(fig, pos, route_nodes):
    # grab the coordinates from pos dict
    x_coords = [pos[node][0] for node in route_nodes]
    y_coords = [pos[node][1] for node in route_nodes]   

    # --- Dynamic placeholder for visited edges (will be replaced by frames)
    visited_trace = go.Scatter(
        x=x_coords, y=y_coords,                    # start empty; frames will fill these
        mode="lines",                  # thick path
        line=dict(width=4, color="red"),            # thicker than background
        name="Visited path"            # legend label
    )

    fig.add_trace(visited_trace)
    return fig

fig = visited_path(fig, pos, route_nodes)
fig.show()

In [19]:
def route_to_timeline(G, route_nodes, speed_m_per_s=1.2, step_seconds=0.3):
    """
    Convert a node route into:
      1) 'segments': metadata for each traversed edge (timings, endpoints, distance)
      2) 'frames':   fine-grained steps for animation (time, x, y, completed_edges so far)

    Example: if weight(u,v)=6m and speed=1.2 m/s, duration for that edge is 5 seconds.
    """
    segments, frames = [], []  # segments = edge chunks; frames = animation samples along edges
    t = 0.0                    # running time in seconds since start
    completed_edges = []       # list of (u,v) that we have fully traversed so far

    # Iterate every consecutive node pair (u->v) along the route
    for u, v in zip(route_nodes[:-1], route_nodes[1:]):
        dist = G[u][v]["weight"]                  # distance for this edge (e.g., meters)
        dur  = dist / max(speed_m_per_s, 1e-9)    # time to traverse edge; guard division by zero
        x0, y0 = G.nodes[u]["x"], G.nodes[u]["y"] # start coordinates
        x1, y1 = G.nodes[v]["x"], G.nodes[v]["y"] # end coordinates
        t0, t1 = t, t + dur                       # start/end timestamps for this edge
        # Record the segment metadata (useful for tooltips, logs, KPIs)
        segments.append(dict(u=u, v=v, dist=dist, t0=t0, t1=t1, x0=x0, y0=y0, x1=x1, y1=y1))

        # Decide how many sub-frames we want along this edge for smooth motion
        n_steps = max(1, int(dur / step_seconds))  # e.g., 5s / 0.3s ≈ 16 frames
        for s in range(n_steps):                    # interpolate position for each sub-frame
            frac = s / n_steps                      # fraction along the edge in [0, 1)
            x = x0 + frac * (x1 - x0)               # linear interpolation for x
            y = y0 + frac * (y1 - y0)               # linear interpolation for y
            # Append a frame: (absolute time, current x, current y, edges finished so far)
            frames.append((t0 + frac * dur, x, y, list(completed_edges)))

        # At the very end of the edge, snap to (x1,y1) and mark this edge as completed
        completed_edges.append((u, v))              # e.g., ('A1','A2')
        frames.append((t1, x1, y1, list(completed_edges)))

        t = t1                                      # advance global time to end of this edge

    return segments, frames                         # return granular animation data


In [20]:
import plotly.graph_objects as go  # Interactive plotting (figures, traces, frames)

def _edge_xy_from(G, pos, edges):
    """
    Convert a set of edges [(u,v), ...] into x and y arrays with Nones to separate segments.
    Example: edges=[('A','B'),('B','C')] -> x=[xA,xB,None,xB,xC,None], y=[yA,yB,None,yB,yC,None]
    """
    ex, ey = [], []
    for u, v in edges:
        x0, y0 = pos[u]        # start coordinate of this edge
        x1, y1 = pos[v]        # end coordinate of this edge
        ex += [x0, x1, None]   # None breaks the line between edges for Plotly
        ey += [y0, y1, None]
    return ex, ey              # arrays ready for a go.Scatter(mode="lines")


In [ ]:
# 1) Build positions from node attributes (x,y) so the plot uses your floor-plan coordinates.
pos = {n: (G.nodes[n]['x'], G.nodes[n]['y']) for n in G.nodes()}


In [50]:
co = pd.read_csv("Customer_Order.csv", sep=";")
co

,codCustomer,orderNumber,orderToCollect,Reference,Size (US),quantity (units),creationDate,waveNumber,operator
0,C0000016,124438,8,8N10W9,9.0,6,19/10/2023 07:18,43175,Operator_1
1,C0000016,124438,7,2T1DJM,13.0,1,19/10/2023 07:18,43175,Operator_1
2,C0000016,124438,6,QVFB0Q,105.0,1,19/10/2023 07:18,43175,Operator_1
3,C0000016,124438,5,W25EZN,13.0,1,19/10/2023 07:18,43175,Operator_1
4,C0000016,124438,4,W25EZN,11.0,1,19/10/2023 07:18,43175,Operator_1
...,...,...,...,...,...,...,...,...,...
122365,C0000747,89481,1,5HQR89,105.0,1,05/01/2023 07:42,33168,Operator_2
122366,C0000747,89480,1,8E9F1P,105.0,1,05/01/2023 07:42,33169,Operator_8
122367,C0000747,89479,1,56TC1Z,6.0,1,05/01/2023 07:42,33168,Operator_2
122368,C0000747,89478,1,6M2FJM,12.0,1,05/01/2023 07:42,33168,Operator_2


In [51]:
co["creationDate"] = pd.to_datetime(co["creationDate"])
co

C:\Users\kilid\AppData\Local\Temp\ipykernel_26872\170930392.py:1: UserWarning:

Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



,codCustomer,orderNumber,orderToCollect,Reference,Size (US),quantity (units),creationDate,waveNumber,operator
0,C0000016,124438,8,8N10W9,9.0,6,2023-10-19 07:18:00,43175,Operator_1
1,C0000016,124438,7,2T1DJM,13.0,1,2023-10-19 07:18:00,43175,Operator_1
2,C0000016,124438,6,QVFB0Q,105.0,1,2023-10-19 07:18:00,43175,Operator_1
3,C0000016,124438,5,W25EZN,13.0,1,2023-10-19 07:18:00,43175,Operator_1
4,C0000016,124438,4,W25EZN,11.0,1,2023-10-19 07:18:00,43175,Operator_1
...,...,...,...,...,...,...,...,...,...
122365,C0000747,89481,1,5HQR89,105.0,1,2023-01-05 07:42:00,33168,Operator_2
122366,C0000747,89480,1,8E9F1P,105.0,1,2023-01-05 07:42:00,33169,Operator_8
122367,C0000747,89479,1,56TC1Z,6.0,1,2023-01-05 07:42:00,33168,Operator_2
122368,C0000747,89478,1,6M2FJM,12.0,1,2023-01-05 07:42:00,33168,Operator_2


In [ ]:
start_date, end_date = "2023-02-01", "2023-03-01"
co_filtered = co.loc[(co["creationDate"] >= start_date) & (co["creationDate"] <= end_date), :].copy()
co_filtered.rename(columns={"creationDate":"creationDateTime"}, inplace=True)
co_filtered["creationDate"] = co_filtered["creationDateTime"].dt.date
co_filtered

In [ ]:
co_filtered.groupby(["creationDate"])["orderNumber"].nunique()

,orderNumber,operator
creationDate,,
2023-02-01,95,3
2023-02-02,269,4
2023-02-03,161,4
2023-02-06,218,5
2023-02-07,245,8
2023-02-08,204,7
2023-02-09,282,6
2023-02-10,127,5
2023-02-13,268,8


In [72]:
co_filtered_by_op = co_filtered.groupby(["creationDate", "operator"])["orderNumber"].nunique().reset_index()
co_filtered_by_op

,creationDate,operator,orderNumber
0,2023-02-01,Operator_17,20
1,2023-02-01,Operator_18,33
2,2023-02-01,Operator_8,46
3,2023-02-02,Operator_17,119
4,2023-02-02,Operator_18,85
...,...,...,...
116,2023-02-28,Operator_20,44
117,2023-02-28,Operator_4,35
118,2023-02-28,Operator_5,9
119,2023-02-28,Operator_6,24


In [ ]:
import plotly.express as px
co_filtered_by_op = co_filtered.groupby(["creationDate", "operator"])["orderNumber"].nunique().reset_index()
# normalizing the work of operators by day
co_filtered_by_op_normalized = co_filtered_by_op.pivot(index="creationDate",columns="operator").sum() / co_filtered_by_op.pivot(index="creationDate",columns="operator").count()
co_filtered_by_op_normalized = co_filtered_by_op_normalized.reset_index()
co_filtered_by_op_normalized = (co_filtered_by_op_normalized
                                .rename(columns={0:"avg orders per day"})
                                .sort_values(by="avg orders per day", ascending=False)
                                )
co_filtered_by_op_normalized["operator"] = "Op_" + co_filtered_by_op_normalized["operator"].str.split("_").str[1]
fig = px.bar(co_filtered_by_op_normalized, x="operator", y="avg orders per day")
# calculate the overall avg
overall_avg = co_filtered_by_op_normalized["avg orders per day"].mean()
# add horizontal line
fig.add_hline(y=overall_avg, line_dash="dash", line_color="red",
              annotation_text="Overall Avg", annotation_position="top left")

In [ ]:
operator = "Operator_17"
co_filtered.loc[co_filtered['operator'] == operator, :]

,codCustomer,orderNumber,orderToCollect,Reference,Size (US),quantity (units),creationDateTime,waveNumber,operator,creationDate
97382,C0000013,98162,2,1JT1XB,9.0,1,2023-02-28 16:32:00,35174,Operator_17,2023-02-28
97383,C0000013,98162,1,6M2FJM,105.0,1,2023-02-28 16:32:00,35174,Operator_17,2023-02-28
97384,C0000013,98161,1,KUFVRI,11.0,2,2023-02-28 16:32:00,35174,Operator_17,2023-02-28
97385,C0000013,98160,2,KUFVRI,13.0,1,2023-02-28 16:32:00,35174,Operator_17,2023-02-28
97386,C0000013,98160,1,6M2FJM,10.0,1,2023-02-28 16:32:00,35174,Operator_17,2023-02-28
...,...,...,...,...,...,...,...,...,...,...
110337,C0000008,93457,1,6M2FJM,11.0,2,2023-02-01 11:56:00,34151,Operator_17,2023-02-01
110338,C0000008,93456,1,6M2FJM,11.0,1,2023-02-01 11:56:00,34151,Operator_17,2023-02-01
110339,C0000008,93455,1,6M2FJM,11.0,1,2023-02-01 11:56:00,34151,Operator_17,2023-02-01
110340,C0000008,93454,3,6M2FJM,105.0,4,2023-02-01 11:56:00,34151,Operator_17,2023-02-01


In [138]:
co_filtered_date_op_order = co_filtered.groupby(["operator", "orderNumber"])["Reference"].nunique().reset_index().sort_values("Reference", ascending=False)
co_filtered_date_op_order

,operator,orderNumber,Reference
3335,Operator_4,94891,22
1108,Operator_18,93981,17
627,Operator_17,96413,16
1455,Operator_18,95279,15
330,Operator_17,94589,14
...,...,...,...
14,Operator_14,95672,1
12,Operator_14,95670,1
20,Operator_14,95714,1
19,Operator_14,95713,1
